In [6]:
import datetime
import pandas as pd
import requests
import time
from typing import Dict, List
import duckdb
import os

In [7]:
SLEEPER_REST_API = "https://api.sleeper.com"

def get_player_week_stats(player_id: str, season: int) -> List[Dict]:
    url = f"{SLEEPER_REST_API}/stats/nfl/player/{player_id}?season_type=regular&season={season}&grouping=week"
    time.sleep(0.1)
    req = requests.get(url)
    res = req.json()
    raw = res.values()
    week_stats = list(filter(lambda v: v is not None, raw))
    return week_stats


def transform_week_stats(raw_week_stats: List[Dict]) -> List[Dict]:    
    return [
        {
            "stat_id": r.get("season") + "_" + str(r.get("week")) + "_" + r.get("player_id"),
            "week_id": r.get("season") + "_" + str(r.get("week")),
            "player_id": r.get("player_id"),
            "game_id": r.get("game_id"),
            "season": r.get("season"),
            "week": r.get("week"),
            "team": r.get("team"),
            "opponent": r.get("opponent"),
            "played_at": int(datetime.datetime.strptime(r.get("date"), "%Y-%m-%d").timestamp() * 1000),
            "receiving_targets": int(r.get("stats", {}).get("rec_tgt", 0)),
            "receiving_catches": int(r.get("stats", {}).get("rec", 0)),
            "receiving_yards": int(r.get("stats", {}).get("rec_yd", 0)),
            "snaps": int(r.get("stats", {}).get("off_snp", 0))
            
        }
        for r in raw_week_stats
    ]

def get_all_players() -> List[Dict]:
    url = f"{SLEEPER_REST_API}/v1/players/nfl"
    req = requests.get(url)
    players_by_id = req.json()
    players = [
        {
            "player_id": p.get("player_id"),
            "name": p.get("full_name"),
            "position": p.get("position"),
            "current_team": p.get("team"),
            "jersey_number": int(p.get("number")) if p.get("number") is not None else None
        }
        for p in players_by_id.values()
    ]
    return players

In [8]:
player_ids = [11628, 10232, 8112, 7090, 9997, 7571, 11637, 8134, 12526, 11626, 4983, 11620, 7564, 6801, 6783, 10444, 6786, 8137, 5045, 9494, 7547, 8148, 12501, 10222, 7569, 4950, 9500, 6819, 11631, 12530, 11624, 10229, 12483, 2374, 11635, 1479, 9493, 2133, 3321, 7526, 6794, 9756, 2449, 9501, 8144, 11834, 11632, 8126, 8146, 11625, 5859, 7525, 5846, 1689, 11638, 7049, 9488, 4039, 2216, 12514, 4981, 12499, 5927, 5872]
all_stats = []


for player in player_ids:
    raw_stats = get_player_week_stats(player_id=player, season=2025)
    stats = transform_week_stats(raw_stats)
    all_stats.extend(stats)

df_stats = pd.DataFrame(all_stats)
df_stats.to_csv("raw_data/weekly_stats.csv", index=False)

df_players = pd.DataFrame(get_all_players())
# df_players = pd.read_csv('players.csv')
df_players['player_id'] = df_players['player_id'].astype(str)
df_players.to_csv("raw_data/players.csv", index=False)

df_wrs = pd.read_csv('raw_data/wrs.csv')
df_wrs = df_wrs.drop(columns=['name'])
df_wrs['player_id'] = df_wrs['player_id'].astype(str)
df_wrs = pd.merge(df_wrs, df_players, on="player_id", how="left")
df_wrs.to_csv("raw_data/wrs.csv", index=False)


In [9]:
CREATE_FROM_CSV_QUERY = """
CREATE TABLE {table_name} AS
SELECT *
FROM read_csv_auto('{filename}')
""".strip()

CREATE_TEMPLATE_BY_FORMAT = {
    "csv": CREATE_FROM_CSV_QUERY,
}


def create_table(con, config: Dict):
    table_name = config.get("name")
    if not table_name:
        raise ValueError("Must specify table name.")

    table_path = config.get("path")
    if not table_path:
        raise ValueError("Must specify table path.")

    table_format = config.get("format")
    if not table_format:
        raise ValueError("Must specify table format.")
    if table_format not in CREATE_TEMPLATE_BY_FORMAT:
        valid_formats = ", ".join(CREATE_TEMPLATE_BY_FORMAT.keys())
        error = f"Invalid format `{table_format}`. Must be one of: {valid_formats}."
        raise ValueError(error)

    template = CREATE_TEMPLATE_BY_FORMAT.get(table_format)
    create_query = template.format(table_name=table_name, filename=table_path)
    con.execute(create_query)


def create_database(database_name: str, table_configs: List[Dict]):
    if os.path.exists(database_name):
        os.remove(database_name)

    with duckdb.connect(database_name) as con:
        for table_config in table_configs:
            create_table(con, table_config)

In [10]:
DATABASE = "database.db"
TABLES_TO_CREATE = [
    {
        "name": "weekly_data",
        "path": "raw_data/weekly_stats.csv",
        "format": "csv",
    },
    {
        "name": "wr_data",
        "path": "raw_data/wrs.csv",
        "format": "csv",
    },
]

create_database(database_name=DATABASE, table_configs=TABLES_TO_CREATE)



In [19]:
teams = ["ARI","ATL","BAL","BUF","CAR","CHI","CIN","CLE","DAL","DEN","DET","GB","HOU","IND","JAX","KC","LV","LAC","LAR","MIA","MIN","NE","NO","NYG","NYJ","PHI","PIT","SEA","SF","TB","TEN","WAS"]

with duckdb.connect(DATABASE) as con:
    for team in teams:
        con.sql(f"""
        SELECT weekly_data.player_id, name, COUNT(game_id) AS games_played, depth_chart_rank
        FROM weekly_data
        JOIN wr_data ON wr_data.player_id = weekly_data.player_id
        WHERE current_team_x = '{team}' AND snaps > 0
        GROUP BY weekly_data.player_id, name, depth_chart_rank
        ORDER BY depth_chart_rank ASC
        ;
        """).show()

┌───────────┬─────────────────┬──────────────┬──────────────────┐
│ player_id │      name       │ games_played │ depth_chart_rank │
│   int64   │     varchar     │    int64     │      int64       │
├───────────┼─────────────────┼──────────────┼──────────────────┤
│     11628 │ Marvin Harrison │           12 │                1 │
│     10232 │ Michael Wilson  │           16 │                2 │
└───────────┴─────────────────┴──────────────┴──────────────────┘

┌───────────┬────────────────┬──────────────┬──────────────────┐
│ player_id │      name      │ games_played │ depth_chart_rank │
│   int64   │    varchar     │    int64     │      int64       │
├───────────┼────────────────┼──────────────┼──────────────────┤
│      8112 │ Drake London   │           11 │                1 │
│      7090 │ Darnell Mooney │           14 │                2 │
└───────────┴────────────────┴──────────────┴──────────────────┘

┌───────────┬────────────────┬──────────────┬──────────────────┐
│ player_id │   

In [13]:
with duckdb.connect(DATABASE) as con:
    
    con.sql("""
        CREATE OR REPLACE TABLE offense_games AS
        SELECT DISTINCT game_id, team, week
        FROM weekly_data;
    """)

    con.sql("""
        SELECT *
        FROM offense_games;
    """).show()


┌───────────┬─────────┬───────┐
│  game_id  │  team   │ week  │
│   int64   │ varchar │ int64 │
├───────────┼─────────┼───────┤
│ 202511333 │ ARI     │    13 │
│ 202511601 │ ARI     │    16 │
│ 202511707 │ ARI     │    17 │
│ 202511101 │ ARI     │    11 │
│ 202511014 │ ATL     │    10 │
│ 202511601 │ ATL     │    16 │
│ 202511324 │ ATL     │    13 │
│ 202510303 │ BAL     │     3 │
│ 202510104 │ BUF     │     1 │
│ 202510224 │ BUF     │     2 │
│     ·     │  ·      │     · │
│     ·     │  ·      │     · │
│     ·     │  ·      │     · │
│ 202511206 │ PIT     │    12 │
│ 202511403 │ PIT     │    14 │
│ 202511828 │ PIT     │    18 │
│ 202511231 │ SF      │    12 │
│ 202511531 │ SF      │    15 │
│ 202510935 │ SEA     │     9 │
│ 202511104 │ TB      │    11 │
│ 202511134 │ TEN     │    11 │
│ 202511735 │ WAS     │    17 │
│ 202510816 │ WAS     │     8 │
├───────────┴─────────┴───────┤
│ 539 rows          3 columns │
│ (20 shown)                  │
└─────────────────────────────┘



In [17]:
with duckdb.connect(DATABASE) as con:
    con.sql("""
        CREATE OR REPLACE TABLE wr1_games AS
        SELECT offense_games.game_id, offense_games.team, offense_games.week, weekly_data.player_id, name, 1 AS wr1_played
        FROM offense_games
        JOIN weekly_data ON weekly_data.game_id = offense_games.game_id AND weekly_data.team = offense_games.team
        JOIN wr_data ON wr_data.player_id = weekly_data.player_id
        WHERE depth_chart_rank = 1 AND snaps > 0 and current_team_x = weekly_data.team
        ORDER BY offense_games.team, offense_games.week, offense_games.game_id
        ;
    """)


    con.sql("""
        CREATE OR REPLACE TABLE wr2_games AS
        SELECT offense_games.game_id, offense_games.team, offense_games.week, weekly_data.player_id, name, 1 AS wr2_played
        FROM offense_games
        JOIN weekly_data ON weekly_data.game_id = offense_games.game_id AND weekly_data.team = offense_games.team
        JOIN wr_data ON wr_data.player_id = weekly_data.player_id
        WHERE depth_chart_rank = 2 AND snaps > 0 and current_team_x = weekly_data.team
        ORDER BY offense_games.team, offense_games.week, offense_games.game_id
        ;
    """)

    con.sql("""
        CREATE OR REPLACE TABLE game_data AS
        SELECT og.game_id, og.team, og.week, MAX(COALESCE(wr1_games.wr1_played, 0)) AS wr1_played, MAX(COALESCE(wr2_games.wr2_played, 0)) AS wr2_played
        FROM offense_games og
        LEFT JOIN wr1_games ON wr1_games.game_id = og.game_id AND wr1_games.team = og.team
        LEFT JOIN wr2_games ON wr2_games.game_id = og.game_id AND wr2_games.team = og.team
        GROUP BY og.game_id, og.team, og.week
        ORDER BY og.team, og.week, og.game_id;
    """)

    con.sql("SELECT * FROM wr1_games").show()
    con.sql("SELECT * FROM wr2_games").show()
    # con.sql("SELECT * FROM game_data").show()


┌───────────┬─────────┬───────┬───────────┬─────────────────┬────────────┐
│  game_id  │  team   │ week  │ player_id │      name       │ wr1_played │
│   int64   │ varchar │ int64 │   int64   │     varchar     │   int32    │
├───────────┼─────────┼───────┼───────────┼─────────────────┼────────────┤
│ 202510122 │ ARI     │     1 │     11628 │ Marvin Harrison │          1 │
│ 202510201 │ ARI     │     2 │     11628 │ Marvin Harrison │          1 │
│ 202510331 │ ARI     │     3 │     11628 │ Marvin Harrison │          1 │
│ 202510401 │ ARI     │     4 │     11628 │ Marvin Harrison │          1 │
│ 202510501 │ ARI     │     5 │     11628 │ Marvin Harrison │          1 │
│ 202510614 │ ARI     │     6 │     11628 │ Marvin Harrison │          1 │
│ 202510701 │ ARI     │     7 │     11628 │ Marvin Harrison │          1 │
│ 202510909 │ ARI     │     9 │     11628 │ Marvin Harrison │          1 │
│ 202511030 │ ARI     │    10 │     11628 │ Marvin Harrison │          1 │
│ 202511333 │ ARI     │  

In [18]:
with duckdb.connect(DATABASE) as con:
    # con.sql("""
    #     SELECT *
    #     FROM weekly_data
    #     JOIN game_data ON game_data.game_id = weekly_data.game_id AND game_data.team = weekly_data.team
    #     WHERE wr1_played = 1 AND wr2_played = 1
    #     ORDER BY weekly_data.team, weekly_data.game_id, weekly_data.week
    #     ;
    # """).show()

    con.sql("""
        SELECT *
        FROM weekly_data
        JOIN game_data ON game_data.game_id = weekly_data.game_id AND game_data.team = weekly_data.team
        WHERE wr1_played = 0 AND wr2_played = 1
        ORDER BY weekly_data.team, weekly_data.game_id, weekly_data.week
        ;
    """).show(max_rows=50)

┌───────────────┬─────────┬───────────┬───────────┬────────┬───────┬─────────┬──────────┬───────────────┬───────────────────┬───────────────────┬─────────────────┬───────┬───────────┬─────────┬───────┬────────────┬────────────┐
│    stat_id    │ week_id │ player_id │  game_id  │ season │ week  │  team   │ opponent │   played_at   │ receiving_targets │ receiving_catches │ receiving_yards │ snaps │  game_id  │  team   │ week  │ wr1_played │ wr2_played │
│    varchar    │ varchar │   int64   │   int64   │ int64  │ int64 │ varchar │ varchar  │     int64     │       int64       │       int64       │      int64      │ int64 │   int64   │ varchar │ int64 │   int32    │   int32    │
├───────────────┼─────────┼───────────┼───────────┼────────┼───────┼─────────┼──────────┼───────────────┼───────────────────┼───────────────────┼─────────────────┼───────┼───────────┼─────────┼───────┼────────────┼────────────┤
│ 2025_11_10232 │ 2025_11 │     10232 │ 202511101 │   2025 │    11 │ ARI     │ SF       